In [2]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *

## Dataset

In [29]:
room_df = pd.read_excel("../data/Admission.xlsx", sheet_name='room')
room_df = room_df[room_df['dataset']==13].drop(columns=['dataset']).reset_index(drop=True)
patient_df = pd.read_excel("../data/Admission.xlsx", sheet_name='patient')
patient_df = patient_df[patient_df['dataset']==13].drop(columns=['dataset']).reset_index(drop=True)

room_df.index=room_df['room ID']
patient_df.index=patient_df['patient ID']

print("--------------------------------------------Room Information-------------------------------------------")
display(room_df)
print("--------------------------------------------Patient Information--------------------------------------------")
display(patient_df)

--------------------------------------------Room Information-------------------------------------------


,room ID,room capacity,department,gender,telemetry,oxygen,nitrogen,television,MiniAge,MaxAge,Dep_Specialism_1,Dep_Specialism_2,Dep_Specialism_3
room ID,,,,,,,,,,,,,
11,11,1,1,D,0,1,0.0,0.0,65,0,1,2,3
12,12,1,1,M,0,0,0.0,0.0,65,0,1,2,3
13,13,1,1,M,1,1,1.0,1.0,65,0,1,2,3
14,14,2,1,D,0,1,1.0,0.0,65,0,1,2,3
15,15,2,1,F,1,0,1.0,1.0,65,0,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,5121,4,5,M,0,1,1.0,1.0,0,0,5,1,2
5122,5122,4,5,N,1,0,1.0,0.0,0,0,5,1,2
5123,5123,4,5,F,0,0,0.0,1.0,0,0,5,1,2


--------------------------------------------Patient Information--------------------------------------------


,patient ID,age,gender,admissionDate,dischargeDate,AssignedSpecialisms,FirstSpecialism,FirstDuration,SecondSpecialism,SecondDuration,roomCapacityPreference,telemetryFirst,oxygenFirst,nitrogenFirst,televisionFirst,telemetrySecond,oxygenSecond,nitrogenSecond,televisionSecond
patient ID,,,,,,,,,,,,,,,,,,,
1,1,88,F,0,9,1,4,9,NaN,NaN,2,1,1,0.0,0.0,0,1,0.0,1.0
2,2,106,M,0,8,1,3,8,NaN,NaN,2,0,0,0.0,1.0,1,1,1.0,1.0
3,3,81,F,0,4,1,1,4,NaN,NaN,2,0,0,0.0,0.0,0,1,1.0,0.0
4,4,28,M,0,8,2,2,6,3.0,2.0,2,1,0,0.0,0.0,1,1,1.0,1.0
5,5,41,M,0,12,2,3,6,2.0,6.0,1,0,0,1.0,0.0,1,1,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,903,104,F,27,36,1,4,9,NaN,NaN,4,0,0,0.0,1.0,1,1,1.0,1.0
904,904,10,F,27,29,1,5,2,NaN,NaN,2,0,0,0.0,1.0,0,1,1.0,1.0
905,905,37,F,27,33,1,3,6,NaN,NaN,1,1,0,0.0,0.0,1,1,0.0,1.0


In [30]:
room_df['MaxAge'] = room_df['MaxAge'].replace(0, 150)
patient_df['gender'] = np.where(patient_df['gender']=='F', 1, 0)

room_df[room_df.Dep_Specialism_1==5].shape[0]#28/125
room_df[room_df.Dep_Specialism_2==5].shape[0]#27/125
room_df[room_df.Dep_Specialism_3==5].shape[0]#28/125
patient_df[patient_df.FirstSpecialism==5].shape[0]#59/907
patient_df[patient_df.SecondSpecialism==5].shape[0]#18/907

room_df = room_df[(room_df.Dep_Specialism_1!=5) & (room_df.Dep_Specialism_2!=5) & (room_df.Dep_Specialism_3!=5)]
patient_df = patient_df[(patient_df.FirstSpecialism!=5) & (patient_df.SecondSpecialism!=5)].loc[:150,:]

###  Constant Variables - Benchmarks
- Specialisms
- Departments
- Rooms
- RoomProperties
- Patients
- Days
- NDays: total number of days
- W: weights for penalties

In [31]:
#Specialisms = [1,2,3,4,5]
Specialisms = [1,2,3,4]
#Departments = [1,2,3,4,5]
Rooms = room_df['room ID'].to_list()
RoomProperties = ['telemetry', 'oxygen', 'nitrogen', 'television']
Patients = patient_df['patient ID'].to_list()
#Days = list(range(min(patient_df.admissionDate), max(patient_df.dischargeDate)+1))
Days = list(range(0, 48))
NDays = len(Days)
W = [5,2,0.8,1,11]

### Constant Variables - Patients
1. Numbers
- P: total number of patients

2. Lists
- Patients: list of patient id
- PF, PM: female patients and male patients
- PG_i: gender of patient i
- PAge_i: age of patient i
- AD_i: admission date of patient i
- DD_i: discharge date of patient i
- PS_ij: =1 if patient i need to go to specialism j
- Duration_ij: length of time patient i need to stay for specialism j
- PC_i: room capacity preferred by patient i
- PRP_Assign_ij: =1 if room property is assigned to patient i (must have)
- PRP_Prefer_ij: =1 if room property is preferred by patient i (good to have)
- $Severity_{i}=\sum \limits _{j} {Specialisms}_{j}*{Duration}_{ij} $: severity of the patient's illness. 

In [32]:
P = patient_df.shape[0]
PF = patient_df[patient_df.gender==1].index.to_list()
PM = patient_df[patient_df.gender==0].index.to_list()
PG = patient_df.gender
PAge = patient_df.age
AD = patient_df.admissionDate
DD = patient_df.dischargeDate
PC = patient_df.roomCapacityPreference

In [33]:
#PS[i,j]: 1 if patient i has specialism j
#Duration[i,j]: n if patient i need to stay n days for specialism j
sp_temp = patient_df[['FirstSpecialism', 'FirstDuration', 'SecondSpecialism', 'SecondDuration']].fillna(0)
PS = dict()
Duration = dict()
for i in Patients:
    sp1, d1, sp2, d2 = sp_temp.loc[i,]
    for j in Specialisms:
        PS[(i,j)] = 0
        Duration[(i,j)] = 0
        if (j == sp1):
            PS[(i,j)] = 1
            Duration[(i,j)] = d1
        elif (j == sp2):
            PS[(i,j)] = 1
            Duration[(i,j)] = d2
            
#PRP_Assign[i,j]: =1 if patient i has to be in a room with property j
#PRP_Prefer[i,j]
rp_assign = patient_df[patient_df.columns[-8:-4]]
rp_prefer = patient_df[patient_df.columns[-4:]]
rp_assign.columns = ['telemetry', 'oxygen', 'nitrogen', 'television']
rp_prefer.columns = ['telemetry', 'oxygen', 'nitrogen', 'television']
PRP_Assign = dict()
PRP_Prefer = dict()
for i in Patients:
    for j in RoomProperties:
        PRP_Assign[(i,j)] = rp_assign.loc[i,j]
        PRP_Prefer[(i,j)] = rp_prefer.loc[i,j]

In [34]:
#PS[i,j]: 1 if patient i has specialism j
#Duration[i,j]: n if patient i need to stay n days for specialism j
sp_temp = patient_df[['FirstSpecialism', 'FirstDuration', 'SecondSpecialism', 'SecondDuration']].fillna(0)
PS = dict()
Duration = dict()
for i in Patients:
    sp1, d1, sp2, d2 = sp_temp.loc[i,]
    for j in Specialisms:
        PS[(i,j)] = 0
        Duration[(i,j)] = 0
        if (j == sp1):
            PS[(i,j)] = 1
            Duration[(i,j)] = d1
        elif (j == sp2):
            PS[(i,j)] = 1
            Duration[(i,j)] = d2
            
#PRP_Assign[i,j]: =1 if patient i has to be in a room with property j
#PRP_Prefer[i,j]
rp_assign = patient_df[patient_df.columns[-8:-4]]
rp_prefer = patient_df[patient_df.columns[-4:]]
rp_assign.columns = ['telemetry', 'oxygen', 'nitrogen', 'television']
rp_prefer.columns = ['telemetry', 'oxygen', 'nitrogen', 'television']
PRP_Assign = dict()
PRP_Prefer = dict()
for i in Patients:
    for j in RoomProperties:
        PRP_Assign[(i,j)] = rp_assign.loc[i,j]
        PRP_Prefer[(i,j)] = rp_prefer.loc[i,j]

Severity={}
for i in Patients:
    Severity[i]=sum(j*Duration[i,j] for j in Specialisms)

### Constant Variables - Rooms
1. Numbers
- R: total number of rooms

2. Lists
- Rooms: list of room id
- RF, RM, RD, RN: room id lists separated according to gender restrictions
- RwS_i: list of room ids with specialism i
- C_i: capacity of room i
- Depart_R_i: department room i belongs to
- AgeL_i: lower bound of age limit of room i
- AgeU_i: upper bound of age limit of room i
- RS_ij: =1 if room i can be used for specialism j
- RRP_ij: =1 if room i has room property j

In [35]:
R = room_df.shape[0]

RF = room_df[room_df.gender=='F'].index.to_list()
RM = room_df[room_df.gender=='M'].index.to_list()
RD = room_df[room_df.gender=='D'].index.to_list()
RN = room_df[room_df.gender=='N'].index.to_list()

RwS1 = room_df[(room_df.Dep_Specialism_1==1) | (room_df.Dep_Specialism_2==1) | (room_df.Dep_Specialism_3==1)].index.to_list()
RwS2 = room_df[(room_df.Dep_Specialism_1==2) | (room_df.Dep_Specialism_2==2) | (room_df.Dep_Specialism_3==2)].index.to_list()
RwS3 = room_df[(room_df.Dep_Specialism_1==3) | (room_df.Dep_Specialism_2==3) | (room_df.Dep_Specialism_3==3)].index.to_list()
RwS4 = room_df[(room_df.Dep_Specialism_1==4) | (room_df.Dep_Specialism_2==4) | (room_df.Dep_Specialism_3==4)].index.to_list()
RwS = {1:RwS1, 2:RwS2, 3:RwS3, 4:RwS4}
C = room_df['room capacity']
Depart_R = room_df.department
AgeL = room_df.MiniAge
AgeU = room_df.MaxAge

#RS[i,j]=1 if room i can be used for specialism j
sp_room = room_df[['Dep_Specialism_1', 'Dep_Specialism_2', 'Dep_Specialism_3']].fillna(0)
RS = dict()
for i in Rooms:
    sp1, sp2, sp3 = sp_room.loc[i,]
    for j in Specialisms:
        RS[(i,j)] = 0
        if ((j == sp1) | (j == sp2) |(j == sp3)):
            RS[(i,j)] = 1
            
#RRP[i,j]=1 if room i has property j
rp = room_df[['telemetry', 'oxygen', 'nitrogen', 'television']]
RRP = dict()
for i in Rooms:
    for j in RoomProperties:
        RRP[(i,j)] = rp.loc[i,j]

## Optimization Model
1. Decision Variables:
- X_ijk: = 1 if patient i is allocated in room j on day k
- ADReal_i, DDReal_i: the real admission and discharge date of patient i
- RG_jk: = 1 if the gender in room j is female on day k (=0 for male) for RDs
- Peni(i=1~6): penalty for not satisfying soft constraints
- t_ijk: =1 if patient transfered from room j on day k
- inHosp_ik: =1 if patient i is in the hospital on day k
- sig: auxiliary binary variable for the formulation of constraint \#1.2

In [36]:
model = gp.Model("Patient Admission Scheduling")

X = model.addVars(Patients, Rooms, Days, vtype=GRB.BINARY, name=["Patient"+str(i)+"InRoom"+str(j)+"OnDay"+str(k) for i in Patients for j in Rooms for k in Days])
Pen1 = model.addVars(Patients, vtype=GRB.CONTINUOUS, lb=0, name='Penalty1')
Pen2 = model.addVars(Patients, vtype=GRB.CONTINUOUS, lb=0, name='Penalty2')
Pen3 = model.addVars(Patients, vtype=GRB.CONTINUOUS, lb=0, name='Penalty3')
Pen4 = model.addVars(Patients, vtype=GRB.CONTINUOUS, lb=0, name='Penalty4')
Pen5 = model.addVars(Patients, vtype=GRB.CONTINUOUS, lb=0, name='Penalty5')
Pen6 = model.addVars(Patients, vtype=GRB.CONTINUOUS, lb=0, name='Penalty6')
ADReal = model.addVars(Patients, vtype=GRB.INTEGER, lb=0, name=['RealAdmissionDatePatient'+str(i) for i in Patients])
DDReal = model.addVars(Patients, vtype=GRB.INTEGER, lb=0,name=['RealDischargeDatePatient'+str(i) for i in Patients])
RG = model.addVars(RD, Days, vtype=GRB.BINARY, name=['GenderofRoom'+str(j)+'OnDay'+str(k) for j in RD for k in Days])
t = model.addVars(Patients, Rooms, Days, vtype=GRB.BINARY, name=['Patient'+str(i)+'TransferFromRoom'+str(j)+'OnDay'+str(k) for i in Patients for j in Rooms for k in Days])
inHosp = model.addVars(Patients, Days, vtype=GRB.BINARY, name=['Patient'+str(i)+'inHospitalonDay'+str(k) for i in Patients for k in Days])
sig = model.addVars(Patients, Days, vtype=GRB.BINARY)
model.setObjective(quicksum(Pen1[i]+Pen2[i]+Pen3[i]+Pen4[i]+Pen5[i]+Pen6[i] for i in Patients), GRB.MINIMIZE)

In [37]:
# 0. Actual admission date cannot be earlier than pre-defined admission date; Actual discharge date cannot be earlier than pre-defined discharge date
for i in Patients:
    model.addConstr(ADReal[i] >= AD[i])
    model.addConstr(DDReal[i] >= DD[i])

In [38]:
# 1. Each patient should be assigned to only one room on a specific day after they admitted
for i in Patients:
    for k in Days:
        model.addConstr(quicksum(X[i,j,k] for j in Rooms) == inHosp[i,k])

# 1.1 Before the actual admission and after the actual discharge, the patient cannot be assigned to any room
for i in Patients:
    for j in Rooms:
        for k in Days:
            model.addConstr((k - ADReal[i]) * X[i,j,k] >= 0)
            model.addConstr((DDReal[i] - k) * X[i,j,k] >= 0)

# 1.2 The patient's daily presence in the hospital during the actual hospital stay is also tracked
# If k>=ADReal or k<=DDReal, then inHosp=1; otherwise 0
M = 10000
for i in Patients:
    for k in Days:
        model.addConstr(k >= ADReal[i] - M*(1 - inHosp[i,k]))
        model.addConstr(k <= DDReal[i] + M*(1 - inHosp[i,k]))
        model.addConstr(k <= ADReal[i] - 0.001 + M*sig[i,k] + M*inHosp[i,k])
        model.addConstr(k >= DDReal[i] + 0.001 - M*(1 - sig[i,k]) - M*inHosp[i,k])

# 2. Each room should respect its capacity limit on each day
for j in Rooms:
    for k in Days:
        model.addConstr(quicksum(X[i,j,k] for i in Patients) <= C[j])

# 3. The age of assigned patient should between the minimum age requirement and the maximum age requirement of the room
for i in Patients:
    for j in Rooms:
        for k in Days:
            model.addConstr((PAge[i] - AgeL[j]) * X[i,j,k] >= 0)
            model.addConstr((AgeU[j] - PAge[i]) * X[i,j,k] >= 0)

# 4. Define transfer, normal admission dates, but unusual X's, all 0
# t[i,j,k] = 1 if patient i transferred from room j on day k, otherwise 0
# Don't define the last day transfer, it will try to be small to be 0
for i in Patients:
    for k in Days[:-1]:
        for j in Rooms:
            model.addConstr(t[i,j,k] >= (X[i,j,k] - X[i,j,k+1]) * inHosp[i,k+1])

# 5. Limit the period of stay, non sequential stay, but x are not zero now
# Not limit in terms of the capacity yet so everyone discharge on 46
for i in Patients:
    model.addConstr(quicksum(X[i,j,k] for j in Rooms for k in Days) >= sum(Duration[i,s] for s in Specialisms))

# 6. Room's gender preference
# 6.1 For the rooms that its gender is dependent on the gender of the first patient
for j in RD:
    for k in Days:
        model.addConstr(sum(X[i,j,k] for i in PF) <= C[j] * RG[j,k])
        model.addConstr(sum(X[i,j,k] for i in PM) <= C[j] * (1 - RG[j,k]))

# 6.2 For the rooms that that only female patients are allowed
for i in Patients:
    for j in RF:
        for k in Days:
            model.addConstr(X[i,j,k] <= PG[i])

# 6.3 For the rooms that that only male patients are allowed
for i in Patients:
    for j in RM:
        for k in Days:
            model.addConstr(X[i,j,k] <= (1 - PG[i]))          

#Extra. Limit the period of stay by specialism; not feasible if we force patients to leave on 47th while satisfying total duration
#for i in Patients:
#    for s in Specialisms:
#        model.addConstr(quicksum(X[i,j,k] for j in RwS[s] for k in Days) >= Duration[i,s])

In [39]:
# Penalty 1: Penalize the situation of which the needed room property is not assigned to the patient.
# Penalty 2: Penalize the situation of which the preferred room property is not assigned to the patient.
for i in Patients:
    not_satisfied_assign = {}
    not_satisfied_prefer = {}
    for j in Rooms:
        not_satisfied_assign[j] = sum(np.subtract([PRP_Assign[i, l] for l in RoomProperties], [PRP_Assign[i, l] and RRP[j, l] for l in RoomProperties]))
        not_satisfied_prefer[j] = sum(np.subtract([PRP_Prefer[i, l] for l in RoomProperties], [PRP_Prefer[i, l] and RRP[j, l] for l in RoomProperties]))
    model.addConstr(Pen1[i] == sum(not_satisfied_assign[j]*W[0]*sum(X[i,j,k] for k in Days) for j in Rooms))
    model.addConstr(Pen2[i] == sum(not_satisfied_prefer[j]*W[1]*sum(X[i,j,k] for k in Days) for j in Rooms))

In [40]:
# Penalty 3: Penalize the situation of which the room preference is less than the room capacity assigned.
# Y3_ij = 1 if room j's capacity does not satisfy patient i's preference, otherwise 0
Y3 = {}
for i in Patients:
    for j in Rooms:
        Y3[i,j] = int(PC[i] < C[j]) 
for i in Patients:
    model.addConstr(Pen3[i] == quicksum(Y3[i,j] * W[2] * X[i,j,k] for j in Rooms for k in Days))

In [41]:
# Penalty 4: Penalize the situation of which the patient do not have the needed specialism.
for i in Patients:
    not_satisfied_sp = {}
    for j in Rooms:
        not_satisfied_sp[j] = sum(np.subtract([PS[i,l] for l in Specialisms], [PS[i,l] and RS[j,l] for l in Specialisms]))
    model.addConstr(Pen4[i] == sum(not_satisfied_sp[j] * W[3] * sum(X[i,j,k] for k in Days) for j in Rooms))

In [42]:
# Penalty 5: Penalize the total number of transfers of each patient because we want to minimize this situation.
for i in Patients:
    model.addConstr(Pen5[i] == quicksum(t[i,j,k] for j in Rooms for k in Days) * W[4])

In [43]:
# Penalty 6: Penalize the situation of delayed admission of each patient according to their delayed days and severity level.
for i in Patients:
    model.addConstr(Pen6[i] == (ADReal[i] - AD[i]) * Severity[i])

In [44]:
model.params.TimeLimit = 5*60
model.update()
model.optimize()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 714897 rows, 567352 columns and 2513121 nonzeros
Model fingerprint: 0x21aafe57
Model has 822822 quadratic constraints
Variable types: 822 continuous, 566530 integer (566256 binary)
Coefficient statistics:
  Matrix range     [8e-01, 1e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 1e+04]
Presolve removed 682063 rows and 278130 columns (presolve time = 5s) ...
Presolve removed 685080 rows and 278130 columns (presolve time = 10s) ...
Presolve removed 690977 rows and 278130 columns (presolve time = 16s) ...
Presolve removed 697011 rows and 278130 columns (presolve time = 20s) ...
Presolve removed 445662 rows and 278130 columns
Presolve time: 23.69s
Presolved: 548983 rows, 429096 

In [45]:
#for v in model.getVars()[:len(Patients)*len(Rooms)*len(Days)]:
for v in model.getVars():
    if v.x != 0:
        print(v.varName,'=',round(v.x))

Patient1InRoom116OnDay1 = 1
Patient1InRoom116OnDay2 = 1
Patient1InRoom116OnDay3 = 1
Patient1InRoom116OnDay4 = 1
Patient1InRoom116OnDay5 = 1
Patient1InRoom116OnDay6 = 1
Patient1InRoom116OnDay7 = 1
Patient1InRoom116OnDay8 = 1
Patient1InRoom116OnDay9 = 1
Patient2InRoom121OnDay1 = 1
Patient2InRoom121OnDay2 = 1
Patient2InRoom121OnDay3 = 1
Patient2InRoom121OnDay4 = 1
Patient2InRoom121OnDay5 = 1
Patient2InRoom121OnDay6 = 1
Patient2InRoom121OnDay7 = 1
Patient2InRoom121OnDay8 = 1
Patient3InRoom14OnDay0 = 1
Patient3InRoom14OnDay1 = 1
Patient3InRoom14OnDay2 = 1
Patient3InRoom14OnDay3 = 1
Patient3InRoom14OnDay4 = 1
Patient4InRoom225OnDay7 = 1
Patient4InRoom225OnDay8 = 1
Patient4InRoom226OnDay2 = 1
Patient4InRoom226OnDay3 = 1
Patient4InRoom226OnDay4 = 1
Patient4InRoom226OnDay5 = 1
Patient4InRoom226OnDay6 = 1
Patient4InRoom234OnDay0 = 1
Patient4InRoom234OnDay1 = 1
Patient5InRoom223OnDay11 = 1
Patient5InRoom223OnDay12 = 1
Patient5InRoom237OnDay1 = 1
Patient5InRoom237OnDay2 = 1
Patient5InRoom237OnDay3

Patient46InRoom238OnDay6 = 1
Patient46InRoom238OnDay7 = 1
Patient46InRoom238OnDay8 = 1
Patient46InRoom238OnDay9 = 1
Patient46InRoom238OnDay10 = 1
Patient47InRoom226OnDay8 = 1
Patient47InRoom229OnDay7 = 1
Patient47InRoom229OnDay9 = 1
Patient47InRoom229OnDay10 = 1
Patient47InRoom241OnDay2 = 1
Patient47InRoom241OnDay3 = 1
Patient47InRoom241OnDay4 = 1
Patient47InRoom241OnDay5 = 1
Patient47InRoom241OnDay11 = 1
Patient47InRoom241OnDay12 = 1
Patient47InRoom242OnDay6 = 1
Patient47InRoom242OnDay13 = 1
Patient47InRoom242OnDay14 = 1
Patient48InRoom225OnDay4 = 1
Patient48InRoom225OnDay5 = 1
Patient48InRoom227OnDay10 = 1
Patient48InRoom227OnDay23 = 1
Patient48InRoom227OnDay24 = 1
Patient48InRoom228OnDay28 = 1
Patient48InRoom228OnDay29 = 1
Patient48InRoom228OnDay30 = 1
Patient48InRoom228OnDay31 = 1
Patient48InRoom228OnDay32 = 1
Patient48InRoom228OnDay33 = 1
Patient48InRoom228OnDay34 = 1
Patient48InRoom228OnDay35 = 1
Patient48InRoom228OnDay36 = 1
Patient48InRoom228OnDay37 = 1
Patient48InRoom228OnDay3

Patient107InRoom117OnDay8 = 1
Patient107InRoom119OnDay3 = 1
Patient107InRoom119OnDay4 = 1
Patient107InRoom119OnDay5 = 1
Patient107InRoom119OnDay6 = 1
Patient107InRoom119OnDay7 = 1
Patient108InRoom231OnDay4 = 1
Patient108InRoom231OnDay5 = 1
Patient108InRoom231OnDay6 = 1
Patient108InRoom231OnDay8 = 1
Patient108InRoom231OnDay9 = 1
Patient108InRoom231OnDay10 = 1
Patient108InRoom236OnDay7 = 1
Patient108InRoom237OnDay11 = 1
Patient109InRoom241OnDay4 = 1
Patient109InRoom241OnDay5 = 1
Patient110InRoom232OnDay4 = 1
Patient110InRoom232OnDay5 = 1
Patient110InRoom232OnDay6 = 1
Patient110InRoom233OnDay7 = 1
Patient110InRoom233OnDay8 = 1
Patient110InRoom233OnDay9 = 1
Patient111InRoom11OnDay4 = 1
Patient111InRoom11OnDay5 = 1
Patient111InRoom11OnDay6 = 1
Patient111InRoom11OnDay7 = 1
Patient111InRoom119OnDay8 = 1
Patient111InRoom119OnDay9 = 1
Patient112InRoom117OnDay7 = 1
Patient112InRoom117OnDay8 = 1
Patient112InRoom118OnDay9 = 1
Patient112InRoom118OnDay10 = 1
Patient113InRoom225OnDay20 = 1
Patient113

Patient128InRoom228OnDay4 = 1
Patient129InRoom224OnDay3 = 1
Patient129InRoom233OnDay5 = 1
Patient129InRoom233OnDay6 = 1
Patient129InRoom233OnDay7 = 1
Patient129InRoom233OnDay8 = 1
Patient129InRoom239OnDay4 = 1
Patient130InRoom117OnDay5 = 1
Patient130InRoom242OnDay6 = 1
Patient131InRoom227OnDay4 = 1
Patient131InRoom227OnDay6 = 1
Patient131InRoom228OnDay5 = 1
Patient131InRoom237OnDay3 = 1
Patient131InRoom237OnDay7 = 1
Patient131InRoom237OnDay8 = 1
Patient132InRoom230OnDay3 = 1
Patient132InRoom230OnDay4 = 1
Patient132InRoom230OnDay5 = 1
Patient132InRoom230OnDay6 = 1
Patient132InRoom230OnDay8 = 1
Patient132InRoom230OnDay9 = 1
Patient132InRoom230OnDay10 = 1
Patient132InRoom230OnDay11 = 1
Patient132InRoom230OnDay12 = 1
Patient132InRoom230OnDay13 = 1
Patient132InRoom230OnDay14 = 1
Patient132InRoom233OnDay7 = 1
Patient132InRoom241OnDay15 = 1
Patient133InRoom232OnDay7 = 1
Patient133InRoom232OnDay8 = 1
Patient133InRoom232OnDay9 = 1
Patient133InRoom233OnDay10 = 1
Patient133InRoom233OnDay11 = 1
Pa

Penalty3[98] = 8
Penalty3[99] = 5
Penalty3[101] = 10
Penalty3[102] = 2
Penalty3[107] = 5
Penalty3[108] = 6
Penalty3[110] = 5
Penalty3[111] = 2
Penalty3[112] = 3
Penalty3[114] = 2
Penalty3[116] = 8
Penalty3[117] = 2
Penalty3[118] = 24
Penalty3[119] = 10
Penalty3[120] = 4
Penalty3[121] = 7
Penalty3[122] = 6
Penalty3[123] = 3
Penalty3[124] = 6
Penalty3[127] = 3
Penalty3[128] = 5
Penalty3[129] = 4
Penalty3[130] = 2
Penalty3[131] = 5
Penalty3[132] = 2
Penalty3[133] = 4
Penalty3[135] = 6
Penalty3[141] = 4
Penalty3[142] = 2
Penalty3[144] = 2
Penalty3[146] = 6
Penalty3[147] = 7
Penalty3[148] = 5
Penalty3[149] = 8
Penalty3[150] = 8
Penalty4[1] = 9
Penalty4[7] = 8
Penalty4[19] = 18
Penalty4[25] = 14
Penalty4[27] = 4
Penalty4[32] = 15
Penalty4[34] = 5
Penalty4[35] = 1
Penalty4[37] = 3
Penalty4[44] = 9
Penalty4[45] = 9
Penalty4[47] = 13
Penalty4[49] = 5
Penalty4[54] = 6
Penalty4[55] = 4
Penalty4[59] = 4
Penalty4[62] = 19
Penalty4[64] = 10
Penalty4[70] = 13
Penalty4[72] = 5
Penalty4[73] = 15
Penalt

...

C562921 = 1
C562922 = 1
C562923 = 1
C562924 = 1
C562925 = 1
C562926 = 1
C562927 = 1
C562928 = 1
C562929 = 1
C562930 = 1
C562931 = 1
C562932 = 1
C562933 = 1
C562934 = 1
C562935 = 1
C562937 = 1
C562938 = 1
C562939 = 1
C562940 = 1
C562941 = 1
C562942 = 1
C562943 = 1
C562944 = 1
C562945 = 1
C562946 = 1
C562947 = 1
C562948 = 1
C562949 = 1
C562950 = 1
C562951 = 1
C562952 = 1
C562953 = 1
C562954 = 1
C562955 = 1
C562956 = 1
C562957 = 1
C562958 = 1
C562959 = 1
C562960 = 1
C562961 = 1
C562962 = 1
C562963 = 1
C562964 = 1
C562965 = 1
C562966 = 1
C562967 = 1
C562968 = 1
C562969 = 1
C562970 = 1
C562971 = 1
C562972 = 1
C562973 = 1
C562974 = 1
C562975 = 1
C562976 = 1
C562977 = 1
C562978 = 1
C562979 = 1
C562980 = 1
C562981 = 1
C562982 = 1
C562983 = 1
C562986 = 1
C562987 = 1
C562988 = 1
C562989 = 1
C562990 = 1
C562991 = 1
C562992 = 1
C562993 = 1
C562994 = 1
C562995 = 1
C562996 = 1
C562997 = 1
C562998 = 1
C562999 = 1
C563000 = 1
C563001 = 1
C563002 = 1
C563003 = 1
C563004 = 1
C563005 = 1
C563006 = 1
C563

C563719 = 1
C563720 = 1
C563721 = 1
C563722 = 1
C563723 = 1
C563724 = 1
C563725 = 1
C563726 = 1
C563727 = 1
C563728 = 1
C563729 = 1
C563730 = 1
C563731 = 1
C563732 = 1
C563733 = 1
C563734 = 1
C563735 = 1
C563736 = 1
C563737 = 1
C563738 = 1
C563739 = 1
C563740 = 1
C563741 = 1
C563742 = 1
C563743 = 1
C563744 = 1
C563745 = 1
C563746 = 1
C563747 = 1
C563748 = 1
C563749 = 1
C563750 = 1
C563751 = 1
C563753 = 1
C563754 = 1
C563755 = 1
C563756 = 1
C563757 = 1
C563758 = 1
C563759 = 1
C563760 = 1
C563761 = 1
C563762 = 1
C563763 = 1
C563764 = 1
C563765 = 1
C563766 = 1
C563767 = 1
C563768 = 1
C563769 = 1
C563770 = 1
C563771 = 1
C563772 = 1
C563773 = 1
C563774 = 1
C563775 = 1
C563776 = 1
C563777 = 1
C563778 = 1
C563779 = 1
C563780 = 1
C563781 = 1
C563782 = 1
C563783 = 1
C563784 = 1
C563785 = 1
C563786 = 1
C563787 = 1
C563788 = 1
C563789 = 1
C563790 = 1
C563791 = 1
C563792 = 1
C563793 = 1
C563794 = 1
C563795 = 1
C563796 = 1
C563797 = 1
C563798 = 1
C563799 = 1
C563801 = 1
C563802 = 1
C563803 = 1
C563

C565150 = 1
C565151 = 1
C565152 = 1
C565153 = 1
C565154 = 1
C565155 = 1
C565156 = 1
C565157 = 1
C565158 = 1
C565159 = 1
C565160 = 1
C565161 = 1
C565162 = 1
C565163 = 1
C565164 = 1
C565165 = 1
C565166 = 1
C565167 = 1
C565168 = 1
C565169 = 1
C565170 = 1
C565171 = 1
C565172 = 1
C565173 = 1
C565174 = 1
C565175 = 1
C565176 = 1
C565177 = 1
C565178 = 1
C565179 = 1
C565180 = 1
C565181 = 1
C565182 = 1
C565183 = 1
C565184 = 1
C565185 = 1
C565186 = 1
C565187 = 1
C565188 = 1
C565189 = 1
C565190 = 1
C565191 = 1
C565194 = 1
C565195 = 1
C565196 = 1
C565197 = 1
C565198 = 1
C565199 = 1
C565200 = 1
C565201 = 1
C565202 = 1
C565203 = 1
C565204 = 1
C565205 = 1
C565206 = 1
C565207 = 1
C565208 = 1
C565209 = 1
C565210 = 1
C565211 = 1
C565212 = 1
C565213 = 1
C565214 = 1
C565215 = 1
C565216 = 1
C565217 = 1
C565218 = 1
C565219 = 1
C565220 = 1
C565221 = 1
C565222 = 1
C565223 = 1
C565224 = 1
C565225 = 1
C565226 = 1
C565227 = 1
C565228 = 1
C565229 = 1
C565230 = 1
C565231 = 1
C565232 = 1
C565233 = 1
C565234 = 1
C565

C566054 = 1
C566055 = 1
C566060 = 1
C566061 = 1
C566062 = 1
C566063 = 1
C566064 = 1
C566065 = 1
C566066 = 1
C566067 = 1
C566068 = 1
C566069 = 1
C566070 = 1
C566071 = 1
C566072 = 1
C566073 = 1
C566074 = 1
C566075 = 1
C566076 = 1
C566077 = 1
C566078 = 1
C566079 = 1
C566080 = 1
C566081 = 1
C566082 = 1
C566083 = 1
C566084 = 1
C566085 = 1
C566086 = 1
C566087 = 1
C566088 = 1
C566089 = 1
C566090 = 1
C566091 = 1
C566092 = 1
C566093 = 1
C566094 = 1
C566095 = 1
C566096 = 1
C566097 = 1
C566098 = 1
C566099 = 1
C566100 = 1
C566101 = 1
C566102 = 1
C566103 = 1
C566109 = 1
C566110 = 1
C566111 = 1
C566112 = 1
C566113 = 1
C566114 = 1
C566115 = 1
C566116 = 1
C566117 = 1
C566118 = 1
C566119 = 1
C566120 = 1
C566121 = 1
C566122 = 1
C566123 = 1
C566124 = 1
C566125 = 1
C566126 = 1
C566127 = 1
C566128 = 1
C566129 = 1
C566130 = 1
C566131 = 1
C566132 = 1
C566133 = 1
C566134 = 1
C566135 = 1
C566136 = 1
C566137 = 1
C566138 = 1
C566139 = 1
C566140 = 1
C566141 = 1
C566142 = 1
C566143 = 1
C566144 = 1
C566145 = 1
C566

C566903 = 1
C566904 = 1
C566905 = 1
C566906 = 1
C566907 = 1
C566908 = 1
C566909 = 1
C566910 = 1
C566911 = 1
C566912 = 1
C566913 = 1
C566914 = 1
C566915 = 1
C566916 = 1
C566917 = 1
C566918 = 1
C566919 = 1
C566927 = 1
C566928 = 1
C566929 = 1
C566930 = 1
C566931 = 1
C566932 = 1
C566933 = 1
C566934 = 1
C566935 = 1
C566936 = 1
C566937 = 1
C566938 = 1
C566939 = 1
C566940 = 1
C566941 = 1
C566942 = 1
C566943 = 1
C566944 = 1
C566945 = 1
C566946 = 1
C566947 = 1
C566948 = 1
C566949 = 1
C566950 = 1
C566951 = 1
C566952 = 1
C566953 = 1
C566954 = 1
C566955 = 1
C566956 = 1
C566957 = 1
C566958 = 1
C566959 = 1
C566960 = 1
C566961 = 1
C566962 = 1
C566963 = 1
C566964 = 1
C566965 = 1
C566966 = 1
C566967 = 1
C566973 = 1
C566974 = 1
C566975 = 1
C566976 = 1
C566977 = 1
C566978 = 1
C566979 = 1
C566980 = 1
C566981 = 1
C566982 = 1
C566983 = 1
C566984 = 1
C566985 = 1
C566986 = 1
C566987 = 1
C566988 = 1
C566989 = 1
C566990 = 1
C566991 = 1
C566992 = 1
C566993 = 1
C566994 = 1
C566995 = 1
C566996 = 1
C566997 = 1
C566

In [46]:
print(len(Patients), len(Rooms), len(Days))

137 42 48


In [47]:
model.objVal

10338.599999999999

In [48]:
schedule = []
for i in Patients:
    patient_i = []
    for k in Days:
        room=float("NAN")
        for j in Rooms:
            varname = f"Patient{i}InRoom{j}OnDay{k}"
            live = model.getVarByName(varname)
            if(live.x == 1):
                room = j
                break
        patient_i.append(room)
    schedule.append(patient_i)
    
schedule = pd.DataFrame(schedule)
schedule.columns = Days
schedule.index = Patients

schedule = schedule.loc[(schedule.notna()).any(axis=1)]
schedule = schedule.loc[:,(schedule.notna()).any(axis=0)]

schedule = schedule.fillna('') 


with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):display(schedule)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
1,,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,,121.0,121.0,121.0,121.0,121.0,121.0,121.0,121.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,14.0,14.0,14.0,14.0,14.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,234.0,234.0,226.0,226.0,226.0,226.0,226.0,225.0,225.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,,237.0,237.0,237.0,237.0,237.0,237.0,237.0,237.0,237.0,237.0,223.0,223.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,,,,16.0,16.0,16.0,16.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,119.0,,,,,
7,,,,,,,240.0,240.0,237.0,226.0,236.0,229.0,229.0,231.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,234.0,234.0,234.0,234.0,234.0,234.0,234.0,234.0,234.0,234.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10,238.0,238.0,238.0,238.0,238.0,238.0,238.0,238.0,238.0,238.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11,121.0,121.0,121.0,121.0,121.0,121.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [49]:
schedule.to_csv('../data/Admission_output_150.csv')

In [55]:
penalties = []
for i in Patients:
    penalty_i = []
    for j in range(1,7):
        varname = f"Penalty{j}[{i}]"
        pen = model.getVarByName(varname)
        penalty_i.append(pen.x)
    penalties.append(penalty_i)

penalties = pd.DataFrame(penalties)
penalties.columns = ['Pen1', 'Pen2', 'Pen3', 'Pen4', 'Pen5', 'Pen6']
penalties.index = Patients

display(penalties)

,Pen1,Pen2,Pen3,Pen4,Pen5,Pen6
1,0.0,18.0,7.2,9.0,0.0,36.0
2,0.0,16.0,6.4,0.0,0.0,24.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,10.0,36.0,1.6,0.0,22.0,0.0
5,10.0,12.0,8.0,0.0,11.0,30.0
...,...,...,...,...,...,...
146,20.0,8.0,6.4,0.0,33.0,8.0
147,0.0,6.0,7.2,10.0,33.0,30.0
148,0.0,10.0,4.8,0.0,22.0,12.0
149,0.0,0.0,8.0,10.0,0.0,0.0


In [63]:
penalties.to_csv('../data/Admission_penalty_150.csv')